Date: 9/25/2024

In [ ]:
# Install correct versions of libraries to ensure compatbility with autograder
!pip install catboost==1.2.7
!pip install category-encoders==2.6.3
!pip install feature-engine==1.8.1
!pip install fonttools==4.54.1
!pip install glmnet-py==0.1.0b2
!pip install lightgbm==4.5.0
!pip install numpy==1.26.4
!pip install pandas==2.2.3
!pip install rpy2==3.5.16
!pip install scikit-learn==1.5.2
!pip install scipy==scipy==1.14.1
!pip install xgboost==2.1

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.9/81.9 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 364.1/364.1 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 378.3/378.3 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 61.5 MB/s eta 0:00:00
  Attempting uninstall: pandas
    Found existing installation: pandas 2.2.2
    Uninstalling pandas-2.2.2:
      Successfully uninstalled pandas-2.2.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.10.1 requires pandas<2.2.3dev0,>=2.0, but you have pandas 2.2.3 which is incompatible.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 2.2.3 which is incompatible.
     ━

In [ ]:
# Connect to drive to access data
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
# Imports
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.linear_model import Ridge, Lasso, ElasticNet
import xgboost as xgb

import warnings
import time
import os
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=UserWarning)

# 1. Exploratory Data Analysis

In [ ]:
# Combining all 10-fold training csvs

dfs = []
folders = [f'fold{i}' for i in range(1, 11)]
base_directory = 'proj1'

for folder in folders:
    folder_path = os.path.join(base_directory, folder)
    os.chdir(folder_path)

    df = pd.read_csv('train.csv')
    dfs.append(df)

combined_df = pd.concat(dfs, ignore_index=True)

In [ ]:
combined_df.head()

,PID,MS_SubClass,MS_Zoning,Lot_Frontage,Lot_Area,Street,Alley,Lot_Shape,Land_Contour,Utilities,...,Fence,Misc_Feature,Misc_Val,Mo_Sold,Year_Sold,Sale_Type,Sale_Condition,Longitude,Latitude,Sale_Price
0,526350040,One_Story_1946_and_Newer_All_Styles,Residential_High_Density,80,11622,Pave,No_Alley_Access,Regular,Lvl,AllPub,...,Minimum_Privacy,NaN,0,6,2010,WD,Normal,-93.619756,42.053014,105000
1,526351010,One_Story_1946_and_Newer_All_Styles,Residential_Low_Density,81,14267,Pave,No_Alley_Access,Slightly_Irregular,Lvl,AllPub,...,No_Fence,Gar2,12500,6,2010,WD,Normal,-93.619387,42.052659,172000
2,527105010,Two_Story_1946_and_Newer,Residential_Low_Density,74,13830,Pave,No_Alley_Access,Slightly_Irregular,Lvl,AllPub,...,Minimum_Privacy,NaN,0,3,2010,WD,Normal,-93.638933,42.060899,189900
3,527105030,Two_Story_1946_and_Newer,Residential_Low_Density,78,9978,Pave,No_Alley_Access,Slightly_Irregular,Lvl,AllPub,...,No_Fence,NaN,0,6,2010,WD,Normal,-93.638925,42.060779,195500
4,527127150,One_Story_PUD_1946_and_Newer,Residential_Low_Density,41,4920,Pave,No_Alley_Access,Regular,Lvl,AllPub,...,No_Fence,NaN,0,4,2010,WD,Normal,-93.633792,42.062978,213500


In [ ]:
combined_df.shape

(20510, 83)

In [ ]:
combined_df.columns

Index(['PID', 'MS_SubClass', 'MS_Zoning', 'Lot_Frontage', 'Lot_Area', 'Street',
       'Alley', 'Lot_Shape', 'Land_Contour', 'Utilities', 'Lot_Config',
       'Land_Slope', 'Neighborhood', 'Condition_1', 'Condition_2', 'Bldg_Type',
       'House_Style', 'Overall_Qual', 'Overall_Cond', 'Year_Built',
       'Year_Remod_Add', 'Roof_Style', 'Roof_Matl', 'Exterior_1st',
       'Exterior_2nd', 'Mas_Vnr_Type', 'Mas_Vnr_Area', 'Exter_Qual',
       'Exter_Cond', 'Foundation', 'Bsmt_Qual', 'Bsmt_Cond', 'Bsmt_Exposure',
       'BsmtFin_Type_1', 'BsmtFin_SF_1', 'BsmtFin_Type_2', 'BsmtFin_SF_2',
       'Bsmt_Unf_SF', 'Total_Bsmt_SF', 'Heating', 'Heating_QC', 'Central_Air',
       'Electrical', 'First_Flr_SF', 'Second_Flr_SF', 'Low_Qual_Fin_SF',
       'Gr_Liv_Area', 'Bsmt_Full_Bath', 'Bsmt_Half_Bath', 'Full_Bath',
       'Half_Bath', 'Bedroom_AbvGr', 'Kitchen_AbvGr', 'Kitchen_Qual',
       'TotRms_AbvGrd', 'Functional', 'Fireplaces', 'Fireplace_Qu',
       'Garage_Type', 'Garage_Yr_Blt', 'Garage_Fi

In [ ]:
# Column types
combined_df.dtypes.value_counts()

,count
object,46
int64,34
float64,3


### Object columns

In [ ]:
# Object type columns
obj_cols = combined_df.select_dtypes(include=['object']).columns
obj_cols

Index(['MS_SubClass', 'MS_Zoning', 'Street', 'Alley', 'Lot_Shape',
       'Land_Contour', 'Utilities', 'Lot_Config', 'Land_Slope', 'Neighborhood',
       'Condition_1', 'Condition_2', 'Bldg_Type', 'House_Style',
       'Overall_Qual', 'Overall_Cond', 'Roof_Style', 'Roof_Matl',
       'Exterior_1st', 'Exterior_2nd', 'Mas_Vnr_Type', 'Exter_Qual',
       'Exter_Cond', 'Foundation', 'Bsmt_Qual', 'Bsmt_Cond', 'Bsmt_Exposure',
       'BsmtFin_Type_1', 'BsmtFin_Type_2', 'Heating', 'Heating_QC',
       'Central_Air', 'Electrical', 'Kitchen_Qual', 'Functional',
       'Fireplace_Qu', 'Garage_Type', 'Garage_Finish', 'Garage_Qual',
       'Garage_Cond', 'Paved_Drive', 'Pool_QC', 'Fence', 'Misc_Feature',
       'Sale_Type', 'Sale_Condition'],
      dtype='object')

In [ ]:
combined_df.select_dtypes(include=['object']).nunique()

,0
MS_SubClass,16
MS_Zoning,7
Street,2
Alley,3
Lot_Shape,4
Land_Contour,4
Utilities,3
Lot_Config,5
Land_Slope,3
Neighborhood,28


### Int64 columns

In [ ]:
# Int64 type columns
int_cols = combined_df.select_dtypes(include=['int64']).columns
int_cols

Index(['PID', 'Lot_Frontage', 'Lot_Area', 'Year_Built', 'Year_Remod_Add',
       'Mas_Vnr_Area', 'BsmtFin_SF_1', 'BsmtFin_SF_2', 'Bsmt_Unf_SF',
       'Total_Bsmt_SF', 'First_Flr_SF', 'Second_Flr_SF', 'Low_Qual_Fin_SF',
       'Gr_Liv_Area', 'Bsmt_Full_Bath', 'Bsmt_Half_Bath', 'Full_Bath',
       'Half_Bath', 'Bedroom_AbvGr', 'Kitchen_AbvGr', 'TotRms_AbvGrd',
       'Fireplaces', 'Garage_Cars', 'Garage_Area', 'Wood_Deck_SF',
       'Open_Porch_SF', 'Enclosed_Porch', 'Three_season_porch', 'Screen_Porch',
       'Pool_Area', 'Misc_Val', 'Mo_Sold', 'Year_Sold', 'Sale_Price'],
      dtype='object')

In [ ]:
# Assessing variability
combined_df.select_dtypes(include=['int64']).describe()

,PID,Lot_Frontage,Lot_Area,Year_Built,Year_Remod_Add,Mas_Vnr_Area,BsmtFin_SF_1,BsmtFin_SF_2,Bsmt_Unf_SF,Total_Bsmt_SF,...,Wood_Deck_SF,Open_Porch_SF,Enclosed_Porch,Three_season_porch,Screen_Porch,Pool_Area,Misc_Val,Mo_Sold,Year_Sold,Sale_Price
count,2.051000e+04,20510.000000,20510.000000,20510.000000,20510.000000,20510.000000,20510.000000,20510.000000,20510.000000,20510.000000,...,20510.000000,20510.000000,20510.000000,20510.000000,20510.000000,20510.000000,20510.000000,20510.000000,20510.000000,20510.000000
mean,7.129976e+08,57.937738,10116.086104,1971.290590,1984.245929,102.273476,4.196782,49.916236,559.265968,1052.791711,...,94.152218,47.178742,23.049439,2.645246,16.180107,2.226329,42.800195,6.210093,2007.791370,181474.696441
std,1.886669e+08,33.389499,7519.625696,30.353249,20.913297,179.812635,2.234877,169.660267,438.635647,435.455948,...,125.560296,66.869586,63.630156,25.756818,56.356702,35.961340,466.409910,2.717650,1.315956,81627.857693
min,5.263011e+08,0.000000,1300.000000,1872.000000,1950.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,2006.000000,12789.000000
25%,5.284581e+08,43.000000,7449.000000,1953.000000,1965.000000,0.000000,3.000000,0.000000,222.000000,792.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,4.000000,2007.000000,129500.000000
50%,5.354521e+08,63.000000,9450.000000,1973.000000,1993.000000,0.000000,3.000000,0.000000,463.000000,990.000000,...,0.000000,27.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.000000,2008.000000,160500.000000
75%,9.071758e+08,79.000000,11577.000000,2001.000000,2004.000000,164.000000,7.000000,0.000000,798.000000,1304.000000,...,168.000000,70.000000,0.000000,0.000000,0.000000,0.000000,0.000000,8.000000,2009.000000,214000.000000
max,1.007100e+09,313.000000,215245.000000,2010.000000,2010.000000,1600.000000,7.000000,1526.000000,2336.000000,6110.000000,...,1424.000000,742.000000,1012.000000,508.000000,576.000000,800.000000,15500.000000,12.000000,2010.000000,755000.000000


### Float64 columns

In [ ]:
# Float64 type columns
float_cols = combined_df.select_dtypes(include=['float64']).columns
float_cols

Index(['Garage_Yr_Blt', 'Longitude', 'Latitude'], dtype='object')

# 2. Preprocessing the data

1. Remove inconsistent rows
2. Drop low-information and unbalanced columns
3. Handle Missing Data
4. Handle unusual observations: winsorization
5. Remove highly-correlated columns
6. Feature Scaling (depending on the model)
7. Handle categorical variables: one-hot encoding for K>2





### 2.1.1 Remove inconsistent rows

In [ ]:
# 1. Year_Built > Year_Remod_Add
combined_df[combined_df['Year_Built'] > combined_df['Year_Remod_Add']][['Year_Built','Year_Remod_Add']]

,Year_Built,Year_Remod_Add
611,2002,2001
2652,2002,2001
4681,2002,2001
6735,2002,2001
10866,2002,2001
12907,2002,2001
14937,2002,2001
16991,2002,2001


In [ ]:
combined_df_cleaned = combined_df[combined_df['Year_Built'] <= combined_df['Year_Remod_Add']]
combined_df_cleaned.shape

(20502, 83)

In [ ]:
# 2. Area = 0 and Materials is not NA
remove_row = combined_df_cleaned[(combined_df_cleaned['Mas_Vnr_Area']==0) & (combined_df_cleaned['Mas_Vnr_Type'].notnull())][['Mas_Vnr_Area', 'Mas_Vnr_Type']].index
remove_row

Index([ 1160,  3220,  3320,  5244,  5343,  7391,  9355,  9426,  9462, 11415,
       13474, 13574, 15499, 15598, 17646, 19610, 19681, 19717],
      dtype='int64')

In [ ]:
combined_df_cleaned = combined_df_cleaned.drop(remove_row, axis=0)
combined_df_cleaned.shape

(20484, 83)

In [ ]:
# 3. Exploring other incosistencies
inconsistent_condition = (
    (combined_df_cleaned['Garage_Area'] == 0) & (
        (~combined_df_cleaned['Garage_Type'].isin(['No_Garage', 'Detchd'])) |
        (combined_df_cleaned['Garage_Yr_Blt'].notna()) |
        (combined_df_cleaned['Garage_Qual'] != 'No_Garage') |
        (combined_df_cleaned['Garage_Cond'] != 'No_Garage') |
        (combined_df_cleaned['Garage_Cars'] != 0) |
        (combined_df_cleaned['Garage_Finish'] != 'No_Garage')
    )
)

inconsistent_rows = combined_df_cleaned[inconsistent_condition]
inconsistent_rows[['Garage_Area', 'Garage_Type', 'Garage_Yr_Blt', 'Garage_Qual',
                  'Garage_Cond', 'Garage_Cars', 'Garage_Finish']]

,Garage_Area,Garage_Type,Garage_Yr_Blt,Garage_Qual,Garage_Cond,Garage_Cars,Garage_Finish


In [ ]:
combined_df_cleaned[(combined_df_cleaned['BsmtFin_SF_1'] == 0) & (combined_df_cleaned['BsmtFin_Type_1'] != 'No_Basement')][['BsmtFin_SF_1', 'BsmtFin_Type_1']]

,BsmtFin_SF_1,BsmtFin_Type_1


In [ ]:
combined_df_cleaned[(combined_df_cleaned['BsmtFin_SF_2'] == 0) & (~combined_df_cleaned['BsmtFin_Type_2'].isin(['No_Basement', 'Unf']))][['BsmtFin_SF_2', 'BsmtFin_Type_2']]

,BsmtFin_SF_2,BsmtFin_Type_2
20,0,BLQ
2071,0,BLQ
4125,0,BLQ
6172,0,BLQ
10275,0,BLQ
12326,0,BLQ
14380,0,BLQ
16427,0,BLQ


In [ ]:
# 4. (?) Verify if BsmtFin_SF_1 + BsmtFin_SF_2 + BsmtUnf_SF = Total_Bsmt_SF
inconsistent_basement = combined_df_cleaned[(combined_df_cleaned['BsmtFin_SF_1'] + combined_df_cleaned['BsmtFin_SF_2'] +
                                    combined_df_cleaned['Bsmt_Unf_SF']) != combined_df_cleaned['Total_Bsmt_SF']]

inconsistent_basement[['BsmtFin_SF_1', 'BsmtFin_SF_2', 'Bsmt_Unf_SF', 'Total_Bsmt_SF']]

,BsmtFin_SF_1,BsmtFin_SF_2,Bsmt_Unf_SF,Total_Bsmt_SF
0,6,144,270,882
1,1,0,406,1329
2,3,0,137,928
3,3,0,324,926
4,3,0,722,1338
...,...,...,...,...
20505,1,0,122,1728
20506,1,0,190,1126
20507,1,0,0,1224
20508,3,0,184,1003


### 2.1.2 Dropping low-information and unbalanced columns

In [ ]:
# Remove low-information and unbalanced columns
drop_cols = ['PID', 'Street', 'Utilities', 'Condition_2', 'Roof_Matl', 'Heating',
             'Pool_QC', 'Misc_Feature', 'Low_Qual_Fin_SF', 'Pool_Area',
             'Longitude','Latitude']
combined_df_cleaned = combined_df_cleaned.drop(columns=drop_cols, axis=1)

In [ ]:
combined_df_cleaned.shape

(20484, 71)

### 2.1.3 Handle Missing Data

In [ ]:
null_counts = combined_df_cleaned.isnull().sum()
null_counts[null_counts > 0]

,0
Mas_Vnr_Type,12409
Garage_Yr_Blt,1084


In [ ]:
# replace Garage_Yr_Blt null values witt 0
combined_df_cleaned['Garage_Yr_Blt'] = combined_df_cleaned['Garage_Yr_Blt'].fillna(0)

In [ ]:
null_counts = combined_df_cleaned.isnull().sum()
null_counts[null_counts > 0]

,0
Mas_Vnr_Type,12409


In [ ]:
# replace 'Mas_Vnr_Type' null values with 'No_MasVnr' because 'Mas_Vnr_Area' for those is 0
combined_df_cleaned['Mas_Vnr_Type'] = combined_df_cleaned['Mas_Vnr_Type'].fillna('No_MasVnr')

In [ ]:
null_counts = combined_df_cleaned.isnull().sum()
null_counts[null_counts > 0]

,0


### 2.1.4 Handle unusual observations: winsorization

In [ ]:
winsorize_cols = ["Lot_Frontage", "Lot_Area", "Mas_Vnr_Area", "BsmtFin_SF_2",
                  "Bsmt_Unf_SF", "Total_Bsmt_SF", "Second_Flr_SF", "First_Flr_SF",
                  "Gr_Liv_Area", "Garage_Area", "Wood_Deck_SF", "Open_Porch_SF",
                  "Enclosed_Porch", "Three_season_porch", "Screen_Porch", "Misc_Val"]

for var in winsorize_cols:
  max_val = combined_df_cleaned[var].quantile(0.95)
  combined_df_cleaned[var] = combined_df_cleaned[var].clip(upper=max_val)

In [ ]:
combined_df_cleaned.shape

(20484, 71)

### 2.1.5 Remove highly-correlated columns

In [ ]:
# Assessing correlation between numerical variables
pearson_corr = combined_df_cleaned.select_dtypes(include=['int64']).corr(method='pearson')
high_corr = pearson_corr[abs(pearson_corr) > 0.7]
filtered_corr = high_corr[high_corr != 1.0].dropna(how='all', axis=0).dropna(how='all', axis=1)
filtered_corr

,Total_Bsmt_SF,First_Flr_SF,Gr_Liv_Area,TotRms_AbvGrd,Garage_Cars,Garage_Area,Sale_Price
Total_Bsmt_SF,NaN,0.791581,NaN,NaN,NaN,NaN,NaN
First_Flr_SF,0.791581,NaN,NaN,NaN,NaN,NaN,NaN
Gr_Liv_Area,NaN,NaN,NaN,0.807827,NaN,NaN,0.707147
TotRms_AbvGrd,NaN,NaN,0.807827,NaN,NaN,NaN,NaN
Garage_Cars,NaN,NaN,NaN,NaN,NaN,0.901504,NaN
Garage_Area,NaN,NaN,NaN,NaN,0.901504,NaN,NaN
Sale_Price,NaN,NaN,0.707147,NaN,NaN,NaN,NaN


Remove one of the highly correlated variables (corr > 0.7): 'First_Flr_SF', 'TotRms_AbvGrd', 'Garage_Cars'

In [ ]:
# Remove low-information, unbalanced, and highly correlated columns
# Under assumption that columns will remain more or less the same across all 10 folds
drop_cols = ['First_Flr_SF', 'TotRms_AbvGrd', 'Garage_Cars']
combined_df_cleaned = combined_df_cleaned.drop(columns=drop_cols, axis=1)

In [ ]:
combined_df_cleaned.shape

(20484, 68)

### 2.1.6 Feature Scaling (depending on the model)

In [ ]:
num_cols = combined_df_cleaned.select_dtypes(include=['int64','float64']).columns
num_cols = num_cols.drop(['Sale_Price', 'Year_Built', 'Year_Remod_Add',
                          'Garage_Yr_Blt', 'Year_Sold', 'Mo_Sold'])

In [ ]:
scaler = StandardScaler()
combined_df_cleaned[num_cols] = scaler.fit_transform(combined_df_cleaned[num_cols])

### 2.1.7 Handle categorical variables: one-hot encoding for K>2

In [ ]:
cat_cols = ['MS_SubClass', 'MS_Zoning', 'Alley', 'Lot_Shape', 'Land_Contour', 'Lot_Config',
            'Land_Slope', 'Neighborhood', 'Condition_1', 'Bldg_Type', 'House_Style',
            'Overall_Qual', 'Overall_Cond', 'Roof_Style', 'Exterior_1st', 'Exterior_2nd',
            'Mas_Vnr_Type', 'Exter_Qual', 'Exter_Cond', 'Foundation', 'Bsmt_Qual',
            'Bsmt_Cond', 'Bsmt_Exposure', 'BsmtFin_Type_1', 'BsmtFin_Type_2', 'Heating_QC',
            'Central_Air', 'Electrical', 'Kitchen_Qual', 'Functional', 'Fireplace_Qu',
            'Garage_Type', 'Garage_Finish', 'Garage_Qual', 'Garage_Cond', 'Paved_Drive',
            'Fence', 'Sale_Type', 'Sale_Condition']

In [ ]:
unique_categories = {col: combined_df_cleaned[col].nunique() for col in cat_cols}
unique_categories_df = pd.DataFrame(list(unique_categories.items()), columns=['column', 'unique_cats'])
unique_categories_df[unique_categories_df['unique_cats']==2]

,column,unique_cats
26,Central_Air,2


In [ ]:
# Case 1: K=2
combined_df_cleaned['Central_Air'].value_counts()

,count
Central_Air,
Y,19077
N,1407


In [ ]:
combined_df_cleaned['Central_Air'] = combined_df_cleaned['Central_Air'].map({'Y': 1, 'N': 0})
combined_df_cleaned['Central_Air'].value_counts()

,count
Central_Air,
1,19077
0,1407


In [ ]:
# Case 2: K>2
dummy_cols = ['MS_SubClass', 'MS_Zoning', 'Alley', 'Lot_Shape', 'Land_Contour', 'Lot_Config',
              'Land_Slope', 'Neighborhood', 'Condition_1', 'Bldg_Type', 'House_Style',
              'Overall_Qual', 'Overall_Cond', 'Roof_Style', 'Exterior_1st', 'Exterior_2nd',
              'Mas_Vnr_Type', 'Exter_Qual', 'Exter_Cond', 'Foundation', 'Bsmt_Qual',
              'Bsmt_Cond', 'Bsmt_Exposure', 'BsmtFin_Type_1', 'BsmtFin_Type_2', 'Heating_QC',
              'Electrical', 'Kitchen_Qual', 'Functional', 'Fireplace_Qu',
              'Garage_Type', 'Garage_Finish', 'Garage_Qual', 'Garage_Cond', 'Paved_Drive',
              'Fence', 'Sale_Type', 'Sale_Condition']

encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore', drop='first')
encoded_data = encoder.fit_transform(combined_df_cleaned[dummy_cols])
encoded_df = pd.DataFrame(encoded_data, columns=encoder.get_feature_names_out(dummy_cols))
combined_df_cleaned_encoded = pd.concat([combined_df_cleaned.reset_index(drop=True), encoded_df], axis=1)
combined_df_cleaned_encoded = combined_df_cleaned_encoded.drop(columns=dummy_cols)

In [ ]:
combined_df_cleaned_encoded.shape

(20484, 270)

## FUNCTON: data_prep()

In [ ]:
def data_prep(train_dir, test_dir, lin_reg = False, tree = False):
  # train_df contains 'Sale_Price'
  train_df = pd.read_csv(train_dir)

  # test_df does not contain 'Sale_Price'
  test_df = pd.read_csv(test_dir)

  # 1. Remove inconsistent rows

    # 1.1 Remove: Year_Built > Year_Remod_Add
  train_df = train_df[train_df['Year_Built'] <= train_df['Year_Remod_Add']]

    # 1.2 Remove: Area = 0 and Materials != NA
  train_idx = train_df[(train_df['Mas_Vnr_Area']==0) & (train_df['Mas_Vnr_Type'].notnull())][['Mas_Vnr_Area', 'Mas_Vnr_Type']].index
  train_df = train_df.drop(train_idx, axis=0)

  # 2. Drop low-information and unbalanced columns
  drop_cols = ['PID', 'Street', 'Utilities', 'Condition_2', 'Roof_Matl', 'Heating',
                'Pool_QC', 'Misc_Feature', 'Low_Qual_Fin_SF', 'Pool_Area',
                'Longitude','Latitude']

  train_df = train_df.drop(columns=drop_cols, axis=1)
  test_df = test_df.drop(columns=drop_cols, axis=1)

  # 3. Handle missing data

    # 3.1 Replace Garage_Yr_Blt null values with 0
  train_df['Garage_Yr_Blt'] = train_df['Garage_Yr_Blt'].fillna(0)
  test_df['Garage_Yr_Blt'] = test_df['Garage_Yr_Blt'].fillna(0)

    # 3.2 Replace 'Mas_Vnr_Type' null values with 'No_MasVnr'
  train_df['Mas_Vnr_Type'] = train_df['Mas_Vnr_Type'].fillna('No_MasVnr')
  test_df['Mas_Vnr_Type'] = test_df['Mas_Vnr_Type'].fillna('No_MasVnr')


  # 4. Handle unusual observations: winsorization
   # Make sure you are wensorizing test data based on train's 95th percentile value

  winsorize_cols = ["Lot_Frontage", "Lot_Area", "Mas_Vnr_Area", "BsmtFin_SF_2",
                    "Bsmt_Unf_SF", "Total_Bsmt_SF", "Second_Flr_SF", "First_Flr_SF",
                    "Gr_Liv_Area", "Garage_Area", "Wood_Deck_SF", "Open_Porch_SF",
                    "Enclosed_Porch", "Three_season_porch", "Screen_Porch", "Misc_Val"]

  for col in winsorize_cols:
    max_val = train_df[col].quantile(0.95)

    train_df[col] = train_df[col].clip(upper=max_val)
    test_df[col] = test_df[col].clip(upper=max_val) # max_val based on train_df

  # 5. Remove highly-correlated columns (LR only)
  if lin_reg:
    # Based on Pearson correlation values explored in EDA
    corr_cols = ['First_Flr_SF', 'TotRms_AbvGrd', 'Garage_Cars']

    train_df = train_df.drop(columns=corr_cols, axis=1)
    test_df = test_df.drop(columns=corr_cols, axis=1)


  # 6. Feature Scaling (LR only)
    num_cols = train_df.select_dtypes(include=['int64','float64']).columns
    # Remove columns that should not be scaled (time related and Y)
    num_cols = num_cols.drop(['Sale_Price', 'Year_Built', 'Year_Remod_Add',
                            'Garage_Yr_Blt', 'Year_Sold', 'Mo_Sold'])
    scaler = StandardScaler()

    train_df[num_cols] = scaler.fit_transform(train_df[num_cols])
    test_df[num_cols] = scaler.transform(test_df[num_cols])


  # 7. Handle categorical variables

    # 7.1 K=2: label encoding
  train_df['Central_Air'] = train_df['Central_Air'].map({'Y': 1, 'N': 0})
  test_df['Central_Air'] = test_df['Central_Air'].map({'Y': 1, 'N': 0})

    # 7.2 K>2: one-hot encoding
  dummy_cols = train_df.select_dtypes(include=['object']).columns

  encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')

  train_encoded = encoder.fit_transform(train_df[dummy_cols])
  train_encoded_df = pd.DataFrame(train_encoded, columns=encoder.get_feature_names_out(dummy_cols))
  train_encoded_df = (pd.concat([train_df.reset_index(drop=True), train_encoded_df], axis=1).
                      drop(columns=dummy_cols))

  # Encoder based on train_df
  test_encoded = encoder.transform(test_df[dummy_cols])
  test_encoded_df = pd.DataFrame(test_encoded, columns=encoder.get_feature_names_out(dummy_cols))
  test_encoded_df = (pd.concat([test_df.reset_index(drop=True), test_encoded_df], axis=1).
                      drop(columns=dummy_cols))

  if lin_reg:
    print("[INFO]: Train data preprocessing for Linear Regression model is done!")
    print("[INFO]: Test data preprocessing for Linear Regression model is done!")

  elif tree:
    print("[INFO]: Train data preprocessing for Tree-based model is done!")
    print("[INFO]: Test data preprocessing for Tree-based model is done!")

  return train_encoded_df, test_encoded_df

# 3. Fit Linear Regression Model

Models tried:

- RidgeCV
- LassoCV
- ElasticNetCV

In [ ]:
train_dir = "proj1/fold1/train.csv"
test_dir = "proj1/fold1/test.csv"
test_y = pd.read_csv("proj1/fold1/test_y.csv")

from sklearn.linear_model import RidgeCV, LassoCV, ElasticNetCV

clean_train_lr, clean_test_lr = data_prep(train_dir, test_dir, lin_reg=True)

y = np.log(clean_train_lr['Sale_Price'].to_numpy())
X = clean_train_lr.drop(columns=['Sale_Price']).to_numpy()
X_test = clean_test_lr.to_numpy()


las_mod = LassoCV()
las_mod.fit(X, y)


selected_features = np.where(las_mod.coef_ != 0)[0]

X_train_selected = X[:, selected_features]
X_test_selected = X_test[:, selected_features]

model = Ridge(alpha=1.0)
model.fit(X_train_selected, y)

y_pred = model.predict(X_test_selected)


test_y = np.log(test_y['Sale_Price'].to_numpy())

rmse = mean_squared_error(test_y, y_pred, squared=False)
print(f"RMSE: {rmse}")

RMSE: 0.17295451462737735


ElasticNet

In [ ]:
# Set Base Directory with all of our folds
baseDir = "proj1/"

# Loop over all folds of data
for curFold in range(1, 11):
  print(f"Starting Fold {curFold}...")
  # Get current time
  start_time = time.time()

  # Init Model
  model = ElasticNet(alpha=0.0001, l1_ratio=0.5, max_iter=10000)

  # Set Directories for our data
  train_dir = os.path.join(baseDir, f"fold{curFold}/train.csv")
  test_dir = os.path.join(baseDir , f"fold{curFold}/test.csv")
  test_y_dir = os.path.join(baseDir , f"fold{curFold}/test_y.csv")

  # Run Data Preprocessing
  clean_train_lr, clean_test_lr = data_prep(train_dir, test_dir, lin_reg=True)

  # Load the test_y
  test_y = np.log(pd.read_csv(test_y_dir)["Sale_Price"].to_numpy())

  # Pull out train_y & log scale it
  y = np.log(clean_train_lr['Sale_Price'].to_numpy())

  # Convert Train data to numpy array
  X = clean_train_lr.drop(columns=['Sale_Price']).to_numpy()

  # Fit model
  model.fit(X, y)

  # Convert Test Data to numpy array
  X_test = clean_test_lr.to_numpy()

  # Predict using Test data & scale it regular scale using exp
  y_pred = model.predict(X_test)

  # Calculate RMSE between log prices & Output
  rmse = mean_squared_error(test_y, y_pred, squared=False)
  print(f"Fold {curFold} RMSE: {rmse:.4f}")
  # Calculate Runtime & Output it
  end_time = time.time()
  runtime = end_time - start_time
  print(f"Fold {curFold} Runtime: {runtime:.3f} seconds")
  print("-----------------------------------------------------------")

Starting Fold 1...
Fold 1 RMSE: 0.1212
Fold 1 Runtime: 3.761 seconds
-----------------------------------------------------------
Starting Fold 2...
Fold 2 RMSE: 0.1187
Fold 2 Runtime: 6.326 seconds
-----------------------------------------------------------
Starting Fold 3...
Fold 3 RMSE: 0.1170
Fold 3 Runtime: 2.412 seconds
-----------------------------------------------------------
Starting Fold 4...
Fold 4 RMSE: 0.1206
Fold 4 Runtime: 2.788 seconds
-----------------------------------------------------------
Starting Fold 5...
Fold 5 RMSE: 0.1120
Fold 5 Runtime: 5.995 seconds
-----------------------------------------------------------
Starting Fold 6...
Fold 6 RMSE: 0.1332
Fold 6 Runtime: 6.718 seconds
-----------------------------------------------------------
Starting Fold 7...
Fold 7 RMSE: 0.1296
Fold 7 Runtime: 3.508 seconds
-----------------------------------------------------------
Starting Fold 8...
Fold 8 RMSE: 0.1205
Fold 8 Runtime: 2.918 seconds
----------------------------

# 4. Fit Tree Based Model

Models tried:
- XGBoost
- CatBoost
- RandomForest

In [ ]:
clean_train_lr, clean_test_lr = data_prep(train_dir, test_dir, tree=True)
y = np.log(clean_train_lr['Sale_Price'].to_numpy())
X = clean_train_lr.drop(columns=['Sale_Price']).to_numpy()
X_test = clean_test_lr.to_numpy()

start_time = time.time()
model = xgb.XGBRegressor(n_estimators=5000, max_depth=6, eta=0.05, subsample=0.5)
model.fit(X, y)
end_time = time.time()
runtime = end_time - start_time
print(runtime)

[INFO]: Train data preprocessing for Tree-based model is done!
[INFO]: Test data preprocessing for Tree-based model is done!
43.62233018875122


CatBoost

In [ ]:
# Set Base Directory with all of our folds
baseDir = "proj1/"

from sklearn.ensemble import RandomForestRegressor
import lightgbm as lgb
import catboost as cb

params = []

# Loop over all folds of data
for curFold in range(1, 11):
  print(f"Starting Fold {curFold}...")
  # Get current time
  start_time = time.time()

  # Init Model
  #model = xgb.XGBRegressor(n_estimators=5000, max_depth=6, eta=0.05, subsample=0.5)
  model = cb.CatBoostRegressor(verbose=0, allow_writing_files=False)
  # Set Directories for our data
  train_dir = os.path.join(baseDir, f"fold{curFold}/train.csv")
  test_dir = os.path.join(baseDir , f"fold{curFold}/test.csv")
  test_y_dir = os.path.join(baseDir , f"fold{curFold}/test_y.csv")

  # Run Data Preprocessing
  clean_train_lr, clean_test_lr = data_prep(train_dir, test_dir, tree=True)

  # Load the test_y
  test_y = np.log(pd.read_csv(test_y_dir)["Sale_Price"].to_numpy())

  # Pull out train_y & log scale it
  y = np.log(clean_train_lr['Sale_Price'].to_numpy())

  # Convert Train data to numpy array
  X = clean_train_lr.drop(columns=['Sale_Price']).to_numpy()

  # Fit model
  model.fit(X, y)

  params.append(model.get_all_params())
  # Convert Test Data to numpy array
  X_test = clean_test_lr.to_numpy()

  # Predict using Test data & scale it regular scale using exp
  y_pred = model.predict(X_test)

  # Calculate RMSE between log prices & Output
  rmse = mean_squared_error(test_y, y_pred, squared=False)
  print(f"Fold {curFold} RMSE: {rmse:.4f}")
  # Calculate Runtime & Output it
  end_time = time.time()
  runtime = end_time - start_time
  print(f"Fold {curFold} Runtime: {runtime:.3f} seconds")
  print("-----------------------------------------------------------")

Starting Fold 1...
Fold 1 RMSE: 0.1117
Fold 1 Runtime: 12.673 seconds
-----------------------------------------------------------
Starting Fold 2...
Fold 2 RMSE: 0.1156
Fold 2 Runtime: 11.412 seconds
-----------------------------------------------------------
Starting Fold 3...
Fold 3 RMSE: 0.1133
Fold 3 Runtime: 8.448 seconds
-----------------------------------------------------------
Starting Fold 4...
Fold 4 RMSE: 0.1147
Fold 4 Runtime: 5.816 seconds
-----------------------------------------------------------
Starting Fold 5...
Fold 5 RMSE: 0.1062
Fold 5 Runtime: 8.461 seconds
-----------------------------------------------------------
Starting Fold 6...
Fold 6 RMSE: 0.1270
Fold 6 Runtime: 6.106 seconds
-----------------------------------------------------------
Starting Fold 7...
Fold 7 RMSE: 0.1300
Fold 7 Runtime: 8.953 seconds
-----------------------------------------------------------
Starting Fold 8...
Fold 8 RMSE: 0.1231
Fold 8 Runtime: 5.987 seconds
--------------------------